In [16]:
import pandas as pd

In [18]:
from Collaborative import *

[(0.7745966692414837, 100906), (0.8749999999999811, 87712), (0.880704845927983, 206534), (1.0, 243929), (1.0, 90049)]
[(1.0, '743464672'), (1.0, '60926317'), (1.0, '157322930X'), (1.0, '373250215'), (1.0, '345427637')]


In [2]:
#bx_ratings = pd.read_csv("../newData/df_ratings.csv")
#bx_books = pd.read_csv("../newData/df_bookss.csv")
#bx_users = pd.read_csv("../newData/df_userss.csv")

In [3]:
import operator
def find_best_books(listofBooks, cnt):    
    book_rate = {}
    book_count = {}
    for book,rate in listofBooks:
        if( book_rate.get(book) == None):
            book_rate[book] = 0
            book_count[book] = 0
        book_rate[book] += float(rate)
        book_count[book] += 1
    best_books = []
    limit = ( max(book_count.iteritems(), key=operator.itemgetter(1))[1] ) * 0.2
    for keys in book_count.iterkeys():
        if( book_count[keys] >= limit):
            best_books.append( ( round(book_rate[keys]/book_count[keys] , 2) , keys))
    #print book_count['439139597']
    cnt = min(cnt, len(best_books) )
    return sorted(best_books, reverse=True)[:cnt]
    

In [4]:
#find_best_books( book_country_map[0][' usa'])

In [5]:
def getUserAgeMap(bx_users):
    map_user_age = {}
    #bx_users = pd.read_csv("newData/df_userss.csv")
    for i,item in bx_users.iterrows():
        map_user_age[ item['User-ID'] ] = item['Age']
    return map_user_age

def getUserCountryMap(bx_users):
    map_user_country = {}
    #bx_users = pd.read_csv("newData/df_userss.csv")
    for i,item in bx_users.iterrows():
        map_user_country[ item['User-ID'] ] = item['Location']
    return map_user_country

In [6]:
def getAgeGroup( age ):
    if( age <= 30):
        return 0
    elif ( age > 30 and age <= 60):
        return 1
    return 2

In [7]:
def getDictionaryOfGroupByBooks(user, bx_ratings, bx_users):
    ageMap = getUserAgeMap(bx_users)
    countryMap = getUserCountryMap(bx_users)
    
    group = getAgeGroup( ageMap[user] )
    country = countryMap[ user ]
    
    #bx_ratings = pd.read_csv("newData/df_ratings.csv")
    listOfBooks = []
    for i,item in bx_ratings.iterrows():
        user = item['User-ID']
        if( countryMap[ user ] == country and getAgeGroup( ageMap[ user ] ) == group ):
            listOfBooks.append( (item['ISBN'], item['Book-Rating']) )
    return listOfBooks

In [8]:
def getRecommendationByDemographic(user, cnt=5):
    bx_ratings = pd.read_csv("../newData/df_ratings.csv")
    bx_books = pd.read_csv("../newData/df_bookss.csv")
    bx_users = pd.read_csv("../newData/df_userss.csv")
    listofBooks = getDictionaryOfGroupByBooks(user, bx_ratings, bx_users)
    recommended_books = find_best_books(listofBooks, cnt)
    return recommended_books

In [9]:
getRecommendationByDemographic(276925)

[(9.0, '751503894'),
 (8.6, '8478884459'),
 (8.33, '8495618605'),
 (8.33, '8477204055'),
 (8.33, '345350499')]

In [29]:
import random

def calculate(pref_users, user, recommendation):
    acsum = 0
    cnt = 0
    for rate,book in recommendation:
        if( pref_users[user].get(book) != None):
            cnt += 1
            acsum += abs( rate - pref_users[user][book])**2
    return (acsum, cnt)
def getAccuracy():
    pref_book, _ = makeDictBooks()
    total_users = set( ratings['User-ID'])
    total_users = random.sample( total_users, 10)
    sumofaccuracy = 0
    denom = 0
    pref_users ,_, users_list ,books_list  = makeDictUser()
    for user in total_users:
        recommendation = getRecommendationByDemographic(user,100)
        x,y = calculate(pref_users, user, recommendation)
        sumofaccuracy += x
        denom += y
    print (sumofaccuracy / denom , denom)
        
    
    

In [31]:
getAccuracy()

(0.8288888888888888, 18)


In [32]:
0.8288888888888888**0.5

0.9104333522498441